In [1]:
import pandas as pd
import os

In [2]:
def getMeanXYZ(df):
    if df.empty:
        return 
    #removing excess data so that data falls in 1 second data intervals
    maxIndx = int(len(df)/100)*100
    df = df.iloc[0:maxIndx]
    meanPerSecondDf = pd.DataFrame()
    for i in range(0,int(maxIndx/100)):
        startIndx = i*100
        endIndx = startIndx+100
        tempdf = df.iloc[startIndx:endIndx]
        
        meanDf = tempdf.mean(axis=0)
        stdDf = tempdf.std()
        
        meanPerSecondDf = meanPerSecondDf.append({'mean_x':meanDf.X, 'std_x':stdDf.X, 'mean_y':meanDf.Y,'std_y':stdDf.Y, 'mean_z':meanDf.Z, 'std_z':stdDf.Z},ignore_index=True)
        
    return meanPerSecondDf


In [3]:
#trying out stuff

#need to do this for all files
#reading the file with no header
df = pd.read_csv("Data/G7NZCJ008056297-Justin-left-hand_wash--1970-01-01-00-59-42.csv", header=None)
#renaming the columns
df = df.rename({0:'time', 1: 'X', 2: 'Y', 3: 'Z'},axis='columns')
meanDf = getMeanXYZ(df)
meanDf

,mean_x,std_x,mean_y,std_y,mean_z,std_z
0,1.143861,4.034328,-5.860074,5.869790,-1.623871,8.433468
1,-1.453334,2.617238,-9.683890,3.536158,-2.292453,3.482914
2,-1.406559,2.024348,-8.981754,2.338669,-1.931787,3.848717
3,-1.401110,0.996393,-9.332277,1.301062,-1.621380,2.564707
4,-1.526238,1.521071,-9.325092,2.191825,-2.330414,2.738219
5,-1.460459,1.275429,-8.966546,2.378334,-2.661789,3.072336
6,-2.089008,2.961476,-5.750849,5.267093,3.376175,7.166364


In [4]:
#main code
dataPath = 'Data/'
handWashSubString = 'left-hand_wash'
notHandWashSubstring = 'left-not_hand_wash'

handWashedDf = pd.DataFrame()
notHandWashedDF = pd.DataFrame()

#Reading through the data folder for filenames
for path in os.listdir(dataPath):
    filepath = dataPath+path
    df = pd.read_csv(filepath, header=None)
    df = df.rename({0:'time', 1: 'X', 2: 'Y', 3: 'Z'},axis='columns')
    meanDf = getMeanXYZ(df)
        
    if handWashSubString in path:
        handWashedDf = handWashedDf.append(meanDf,ignore_index=True)
    else:
        notHandWashedDF = notHandWashedDF.append(meanDf,ignore_index=True)

In [5]:
handWashedDf['Activity'] = 'hand_wash'
handWashedDf

,mean_x,std_x,mean_y,std_y,mean_z,std_z,Activity
0,1.143861,4.034328,-5.860074,5.869790,-1.623871,8.433468,hand_wash
1,-1.453334,2.617238,-9.683890,3.536158,-2.292453,3.482914,hand_wash
2,-1.406559,2.024348,-8.981754,2.338669,-1.931787,3.848717,hand_wash
3,-1.401110,0.996393,-9.332277,1.301062,-1.621380,2.564707,hand_wash
4,-1.526238,1.521071,-9.325092,2.191825,-2.330414,2.738219,hand_wash
...,...,...,...,...,...,...,...
128,1.887910,3.164214,-9.568007,4.700721,-1.564834,7.212037,hand_wash
129,1.239794,2.918980,-8.446024,4.624848,-1.580210,6.228527,hand_wash
130,1.422917,3.834809,-8.642763,5.729237,-1.267648,8.845801,hand_wash
131,1.611082,3.741903,-7.173060,7.803930,4.281505,4.671116,hand_wash


In [6]:
notHandWashedDF['Activity'] = 'no_hand_wash'
notHandWashedDF

,mean_x,std_x,mean_y,std_y,mean_z,std_z,Activity
0,-0.900911,3.634586,-3.635065,9.057617,5.996925,5.036683,no_hand_wash
1,-1.366143,17.285437,-2.451123,10.922059,2.033791,5.739746,no_hand_wash
2,3.903151,13.017041,-4.445096,12.318869,-1.307046,9.980050,no_hand_wash
3,-0.702771,9.970328,1.895251,8.044758,1.361341,14.441227,no_hand_wash
4,-4.277816,13.409194,5.746885,5.049177,4.490362,5.652351,no_hand_wash
...,...,...,...,...,...,...,...
347,2.102648,19.781258,5.244063,19.600483,1.813557,9.984353,no_hand_wash
348,7.019203,12.753376,1.703817,10.565443,2.915338,7.825563,no_hand_wash
349,3.025951,18.757725,9.538237,16.932889,4.862440,12.645870,no_hand_wash
350,3.560867,20.552488,12.081628,17.084694,2.858827,12.931070,no_hand_wash


In [9]:
nameOfTheFile = 'processedDataAssignment1.csv'
to_save_df = pd.DataFrame()
to_save_df = to_save_df.append(handWashedDf,ignore_index=True)
to_save_df = to_save_df.append(notHandWashedDF,ignore_index=True)
to_save_df.to_csv(nameOfTheFile, index=False,encoding='utf-8')